In [1]:
from collections import Counter
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import datetime
import json

In [3]:
from rating_api.tournaments import get_tournament_results, get_tournaments_for_release, get_tournaments, get_all_tournaments

In [19]:
from release_procs.tools import calc_c, calculate_bonus_predictions, calc_score_real
from release_procs.calc_rating import calc_bonus

In [4]:
%%time
ttt = get_all_tournaments()

CPU times: user 3.25 s, sys: 140 ms, total: 3.39 s
Wall time: 1min 5s


In [27]:
get_tournament_results('5752', recaps=True, rating=True)[0]

{'team': {'id': 51514,
  'name': 'Язык Чау-Чау',
  'town': {'id': 285, 'name': 'Санкт-Петербург'}},
 'mask': '111111111111111111111111011111111111',
 'current': {'name': 'Язык Чау-Чау',
  'town': {'id': 285, 'name': 'Санкт-Петербург'}},
 'questionsTotal': 35,
 'synchRequest': {'id': 72698, 'venue': {'id': 3505, 'name': 'Чайковский'}},
 'rating': {'inRating': True,
  'b': 1985,
  'predictedPosition': 6,
  'rt': 8351,
  'rb': 7987,
  'rg': 7968,
  'r': 7621,
  'bp': 1861,
  'd1': 124,
  'd2': 121,
  'd': 123},
 'position': 1,
 'controversials': [{'id': 115738,
   'questionNumber': 25,
   'answer': 'вощёная бумага',
   'issuedAt': '2019-12-01T14:24:54+03:00',
   'status': 'D',
   'comment': 'Фактически неверный ответ',
   'resolvedAt': '2019-12-01T14:44:28+03:00',
   'appealJuryComment': None}],
 'flags': [{'id': 11, 'shortName': 'Б'}],
 'teamMembers': [{'flag': 'Л',
   'rating': 13733,
   'player': {'id': 7008,
    'name': 'Алексей',
    'patronymic': 'Владимирович',
    'surname': 'Гилё

In [5]:
len(ttt)

5389

In [8]:
# tournaments_for_release = get_tournaments_for_release(datetime.datetime(2019, 12, 5))

# tournaments_results_for_release = {t['idtournament']: get_tournament_results_by_id(t['idtournament']) 
#                                    for t in tournaments_for_release}

# with open('tournament_results.json', 'w') as f:
#     json.dump(tournaments_results_for_release, f)

In [6]:
with open('data/tournament_results.json') as f:
    tournaments_results_for_release = json.load(f)

In [10]:
release = pd.read_csv('data/release-2019-11-28.csv')
release['Место'] = release['Место'].map(lambda x: x.replace(',', '.')).astype(float)
c = calc_c(release['Рейтинг'][release['Место'] == release['Место'].astype(int)].values)
print(c)

0.11942999030081372


In [23]:
tourn_pd = pd.DataFrame(tournaments_results_for_release['5752'], dtype='float').set_index('idteam')
tourn_pd = tourn_pd.loc[:, ['position', 'tech_rating_rg']]
tourn_pd.columns = ['position', 'rg']
tourn_pd['n_legs'] = 0

In [25]:
%%timeit
tourn_pd.sort_values(by='rg', ascending=False, inplace=True)
tourn_pd['score_pred'] = calculate_bonus_predictions(tourn_pd.rg.values, c=c)
tourn_pd['score_real'] = calc_score_real(tourn_pd.score_pred.values, tourn_pd.position.values)
tourn_pd['bonus'] = calc_bonus(tourn_pd)

12.1 ms ± 384 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
tourn_pd

,position,rg,n_legs
idteam,,,
51514.0,1.0,7968.0,0
5521.0,3.0,5549.0,0
42483.0,3.0,7610.0,0
51739.0,3.0,7140.0,0
4622.0,6.0,7377.0,0
...,...,...,...
75253.0,957.5,0.0,0
75349.0,957.5,0.0,0
68334.0,962.0,0.0,0


In [22]:
%%timeit
calculate_bonus_predictions(calc_rg(tournaments_results_for_release['5752'], 0, 0), c=0.1194)

NameError: name 'calc_rg' is not defined

In [63]:
calculate_rating_release(Counter(), Counter(), [])

Counter()